### srt to txt

In [2]:
import pysrt
import os 
import string
from os import path
import enchant
from guess_language import guess_language
from itertools import compress
import time 
from datetime import datetime
import pandas as pd
from mafan import simplify

## Define some functions first 

In [3]:
##############################
#### define some functions ###
##############################
def check(f):
    filename, file_extension = os.path.splitext(f)
    if file_extension == '.srt':
        return True
    else:
        return False

def check_alignment(ele):
    t2 = ele.text.split('\n')[0]  ## get the first element, see if it is a time stamp
    if '-->' in t2:
        t1_end = ele.end.seconds
        start,end = t2.split(' --> ')
        end = time.strptime(end,'%H:%M:%S,%f')
        t2_end = end.tm_sec
        #print(t1_end," -- ",t2_end)
        if abs(t1_end - t2_end) > 3:
            return False
        else:
            return True
    else:
        return True
    
def get_text_line(ele):
    if check_alignment(ele) is False:
        return False
    try:
        text_list = ele.text.split('\n')
        languages = [guess_language(l) for l in text_list]
        en_index = [l=='en' for l in languages]
        zh_index = [l=='zh' for l in languages]
        en_lines = list(compress(text_list,en_index))
        zh_lines = list(compress(text_list,zh_index))
        if len(en_lines)==0 or len(zh_lines)==0:
            return None
        en_text = ' '.join(en_lines)
        zh_text = simplify(' '.join(zh_lines))
        text_line = en_text + ' | ' + zh_text
        return text_line
    except:
        return None

def get_doc_txt(subs):
    doc = []
    for ele in subs:
        clean_text = get_text_line(ele)
        if clean_text == False:
            return None
        elif clean_text is not None:
            doc.append(clean_text)
    return doc 

def check_last_line(doc):
    en,zh = doc[-1].split('|')
    return len(en)/len(zh)

In [4]:
data_folder = 'data/cn_tw/data/'
result_folder = 'data/cn_tw/result_folder/'
problem_folder = 'data/cn_tw/problem/'

In [5]:
## check if extension is .srt
srts = [data_folder + f for f in os.listdir(data_folder) if check(f)]
len(srts)

2183

In [ ]:
srts[0]

'data/cn_tw/data/Deborah Rhode：一項更有效找到3倍多乳癌偵測技術的發明故事，以及它至今鮮為人知的原因.srt'

In [ ]:
doc_list ={'title':[],'last':[],'status':[]}

for index,srt in enumerate(srts):
    print (len(srts),index+1)
    subs = pysrt.open(srt)
    doc = get_doc_txt(subs)
    fname = srt.split('/')[-1].split('.')[-2]+ '.txt'
    doc_list['title'].append(fname)
    if doc is None:
        doc_list['last'].append('Failed')
        doc_list['status'].append('Failed')
    else:
        with open(result_folder + fname, mode='wt', encoding='utf-8') as myfile:
            myfile.write('\n'.join(doc))
        doc_list['last'].append(check_last_line(doc))
        doc_list['status'].append('Success')

2183 1
2183 2
2183 3
2183 4
2183 5
2183 6
2183 7
2183 8
2183 9
2183 10
2183 11
2183 12
2183 13
2183 14
2183 15
2183 16
2183 17
2183 18
2183 19
2183 20
2183 21
2183 22
2183 23
2183 24
2183 25
2183 26
2183 27
2183 28
2183 29
2183 30
2183 31
2183 32
2183 33
2183 34
2183 35
2183 36
2183 37
2183 38
2183 39
2183 40
2183 41
2183 42
2183 43
2183 44
2183 45
2183 46
2183 47
2183 48
2183 49
2183 50
2183 51
2183 52
2183 53
2183 54
2183 55
2183 56
2183 57
2183 58
2183 59
2183 60
2183 61
2183 62
2183 63
2183 64
2183 65
2183 66
2183 67
2183 68
2183 69
2183 70
2183 71
2183 72
2183 73
2183 74
2183 75
2183 76
2183 77
2183 78
2183 79
2183 80
2183 81
2183 82
2183 83
2183 84
2183 85
2183 86
2183 87
2183 88
2183 89
2183 90
2183 91
2183 92
2183 93
2183 94
2183 95
2183 96
2183 97
2183 98
2183 99
2183 100
2183 101
2183 102
2183 103
2183 104
2183 105
2183 106
2183 107
2183 108
2183 109
2183 110
2183 111
2183 112
2183 113
2183 114
2183 115
2183 116
2183 117
2183 118
2183 119
2183 120
2183 121
2183 122
2183 123
2

In [ ]:
## export log file to csv
df = pd.DataFrame(doc_list)
df.to_csv('ted_txt.csv')

In [ ]:
df_problem=df[df.status!='Failed']
df_problem['last'] = df_problem['last'].astype(float)

In [ ]:
p_files = df_problem[(df_problem['last']>7) | (df_problem['last']<1)].title.values

In [ ]:
for p in p_files:
    print(p)
    os.rename(result_folder+p, problem_folder + p)

In [ ]:
######################################################
### no longer use this, used guess_language instead###
### this is for future reference                   ###
######################################################

# ### ehck each line, see if they are time header, or english or chinese
# def is_english(word):
#     try:
#         return word.encode('ascii').isalpha()
#         en = [e for e in word if word.encode('ascii').isalpha()]
#     except:
#         return False
    
# def is_chinese(word):
#     ch = [c for c in word if u'\u4e00' <= c <= u'\u9fff']
#     if len(ch) > 0:
#         return True
#     else:
#         return False

# def check_language(line):
#     if '-->' in line:
#         return 'tm'  ## time header
#     elif is_english(line):
#         return 'en'
#     elif is_chinese(line):
#         return 'cn'
#     else:
#         return None